In [ ]:
# Load package 
import numpy as np

<h1> Exercise 1: The Simplest Consumption Model (Cake Eating Problem)

Consider the simplest consumption-saving model:

$$\begin{align*}
    V_1^*(W) &= \max_{c_1,c_2,\dots,c_T} \{ \sqrt{c_1} +\beta \sqrt{c_2}+\beta^2\sqrt{c_3}+\cdots+\beta^T\sqrt{c_T} \} \\
    & \text{s.t.} \\
    W &= c_1 + c_2+\cdots+c_T \\
    c_t &\in \mathbb{N}_0, \, \forall t
\end{align*}$$
for $T=3$, $\beta =0.9$ and $W=5$.

We can express this problem with a bellman equation:

$$\begin{align*}
    V_t^*(W_t) &= \max_{0\leq c_t \leq W_t} \{ \sqrt{c_t} +\beta V_{t+1}(W_{t+1}) \} \\
    & \text{s.t.} \\
    W_{t+1} &= W_t -c_t\\
    c_t &\in \mathbb{N}_0, \, \forall t
\end{align*}$$

Note that in the last period $V_T(W_T) = \sqrt{c_T}$ because there is no tomorrow.

### 1. Solve the model using backwards induction: 

In [ ]:
# 1. Setup 
beta = 0.9
W = 5
T = 3

# 2. Initialize
Vstar_bi = np.nan+np.zeros([W+1,T])
Cstar_bi = np.nan + np.zeros([W+1,T])
Cstar_bi[:,T-1] = np.arange(W+1) 
Vstar_bi[:,T-1] = np.sqrt(Cstar_bi[:,T-1])
# 3. solve

# Loop over periods
for t in range(T-2, -1, -1):  #from period T-2, until period 0, backwards  
    
    #loop over states
    for w in range(W+1):
        c = np.arange(w+1)
        w_c = w - c
        V_next = Vstar_bi[w_c,t+1]
        V_guess = np.sqrt(c)+beta*V_next
        Vstar_bi[w,t] = np.amax(V_guess)
        Cstar_bi[w,t] = np.argmax(V_guess)
        
print(Cstar_bi)

### 2. Fill in Exercise_1.py with a function with $\beta$, $W$ and $T$ as inputs

### 3. Import the function and check that it works 

Notice, C is our policy function. The policy function tells us the optimal way of spending our resources in a given period. 

In [ ]:
from Exercise_1 import solve_backwards

beta = 0.9
W = 5
T = 3
C,V = solve_backwards(beta=beta, W=W, T=T)

### 4. Use the policy function to simulate how 5 resources now can be spend optimally over 3 periods.    

In [ ]:
# 4. simulate
C_backwards = np.empty(T)
W_now = W

for t in range(T):
    W_now = int(W_now)   # change the type to integreger 0,1,2,3 and so on
    C_backwards[t] = C[W_now,t]  # Use the solution you find above 
    W_now = W_now-C_backwards[t]

print(C_backwards)

### 5. Solve the model for $T = 10$.

In [ ]:
# 1. Setup 
beta = 0.9
W = 5
T = 10

# 2. - 3. Solve 
C10, V10 = solve_backwards(beta,W,T)  # call function

Simulate $C_1$, $C_2$,...,$C_{10}$ using the policy function found in 5) and W=5

In [ ]:
# define function
def simulate(Cstar,T,W):
    C_backwards = np.empty(T)
    W_now = W
    for t in range(T):
        W_now = int(W_now)   # change the type to integreger 0,1,2,3 and so on
        C_backwards[t] = Cstar[W_now,t]  # I use the solution algorithm 
        W_now = W_now-C_backwards[t]
    return C_backwards


# 4. Simulate
C_sim = simulate(C10,T,W)  # Call function
        
print('Solution for C with T =',T,'and W =', W , ': C =',C_sim)

Play around with W, $\beta$ and see the results. Are the results inline with your intuition?